In [1]:
# Packages
import pandas as pd
import numpy as np
# plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Modelling
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import accuracy_score, cohen_kappa_score, make_scorer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
# import graphviz
# Scaling with Minmax-scaler
from sklearn.preprocessing import MinMaxScaler

# from ydata_profiling import ProfileReport

# import custom functions
from custom_functions import our_metrics

pd.set_option('display.max_rows', 25)

# # mlflow
# import mlflow
# from mlflow.models import infer_signature
# import mlflow.sklearn

# mlflow.set_tracking_uri("http://localhost:5000")
#import logging

#logging.basicConfig(level=logging.WARN)
#logger = logging.getLogger(__name__)


In [2]:
# export processed data set to csv

# export features for tree-based models
X_train_comb = pd.read_csv('../data/petfinder-adoption-prediction/train/X_train_minmax_scaled_processed.csv')
X_test_comb =pd.read_csv('../data/petfinder-adoption-prediction/train/X_test_minmax_scaled_processed.csv')


# export target
y_train_comb = pd.read_csv('../data/petfinder-adoption-prediction/train/y_train.csv')
y_test_comb = pd.read_csv('../data/petfinder-adoption-prediction/train/y_test.csv')


In [3]:
gbc = GradientBoostingClassifier(n_estimators=200,subsample=1, max_leaf_nodes=31, max_features='log2', max_depth=5, loss = 'log_loss', learning_rate=0.025, random_state=42)

In [4]:
gbc.fit(X_train_comb,y_train_comb)

/Users/janschlautmann/neuefische/ds_capstone_pet_adoptability/.venv/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(learning_rate=0.025, max_depth=5,
                           max_features='log2', max_leaf_nodes=31,
                           n_estimators=200, random_state=42, subsample=1)

In [5]:
df = pd.read_csv('../data/petfinder-adoption-prediction/eda/df_eda.csv')
df = df.query('quantity == 1')

In [21]:
df_noy = df.drop("adoptionspeed",axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_noy, df["adoptionspeed"], random_state=42, stratify=df["adoptionspeed"])

df_comb = X_test.reset_index()

# Error Analysis

In [22]:
df_comb["adoptionspeed"] = y_test_comb
df_comb["as_pred"] = gbc.predict(X_test_comb)
df_comb["pred_correct"] = (df_comb["as_pred"] == df_comb["adoptionspeed"]).astype(int)



In [24]:
df_comb.columns

Index(['index', 'type', 'gender', 'maturitysize', 'furlength', 'sterilized',
       'health', 'quantity', 'photoamt', 'color_pattern', 'breed_type',
       'vaccinated_dewormed', 'fee_bin', 'age_bin', 'photoamt_11',
       'description_char', 'adoptionspeed', 'as_pred', 'pred_correct'],
      dtype='object')

In [40]:
tdf = df_comb.groupby('type')["pred_correct"].mean().reset_index()
tdf.insert(0, "feature", tdf.columns[0])

In [41]:
tdf

,feature,type,pred_correct
0,type,0,0.437129
1,type,1,0.371476


In [45]:
# DESCRIPTION LENGTH DROPPED!!!

prop_df = pd.DataFrame(columns=["feature",'group', "prop_correct"])
for cat in ['type', 'gender', 'maturitysize', 'furlength', 'sterilized',
    'health', 'quantity', 'color_pattern', 'breed_type',
    'vaccinated_dewormed', 'fee_bin', 'age_bin', 'photoamt_11']:
    # print(ea_data.groupby(cat)["bank_correct"].mean().reset_index())
    # print("_________")
    test_df = df_comb.groupby(cat)["pred_correct"].mean().reset_index()
    test_df.insert(0, "feature", test_df.columns[0])
    test_df.columns = ["feature","group", "prop_correct"]
    prop_df = pd.concat([prop_df, test_df], axis=0, ignore_index=True)

prop_df.sort_values("prop_correct")

,feature,group,prop_correct
38,photoamt_11,7.0,0.294118
40,photoamt_11,9.0,0.314286
39,photoamt_11,8.0,0.326087
37,photoamt_11,6.0,0.344262
36,photoamt_11,5.0,0.369973
...,...,...,...
29,age_bin,2,0.507143
30,age_bin,3,0.508475
14,health,1,0.532110
31,photoamt_11,0.0,0.681818
